# Income Evaluation - Part 1: Data Cleaning

## 1. Importing Dataset and Necessary Packages

In [1]:
import numpy as np                           # linear algebra
import os                                    # package used to set work directory
import pandas as pd                          # package used to create dataframes
import re                                    # regular expressions

from operator import itemgetter              # package used for sorted, max, min 

In [2]:
data = pd.read_csv('income_evaluation.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              32561 non-null  int64 
 1    workclass       32561 non-null  object
 2    fnlwgt          32561 non-null  int64 
 3    education       32561 non-null  object
 4    education-num   32561 non-null  int64 
 5    marital-status  32561 non-null  object
 6    occupation      32561 non-null  object
 7    relationship    32561 non-null  object
 8    race            32561 non-null  object
 9    sex             32561 non-null  object
 10   capital-gain    32561 non-null  int64 
 11   capital-loss    32561 non-null  int64 
 12   hours-per-week  32561 non-null  int64 
 13   native-country  32561 non-null  object
 14   income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


We have 15 columns, 32,561 rows and, fortunately, no missing values. Some of the columns are completely numerical. An interesting thing is that the numerical values are all integer, which indicates that some of these have actually been rounded to the nearest integers. According to the source, the rest of the columns consist of string values. One of the columns has a name, `fnlwgt`, that doesn't really explain what it is and it is not explained in the source. Some googling teaches us that this should be seen as a weight. The number in this column respresents the number of people in the population for which the data hold. The source of the data set says that the column `income` is binary: it's either '<=50k' or '>50k'. The column names have a space before the name except for `age`. That should be addressed first.

## 2. Data Cleaning

In [4]:
df = data.copy()

We'll start by fixing the column names.

In [5]:
df.columns = [col_name.strip(' ').replace('-', '_') for col_name in df.columns.to_list()]
print(df.columns)

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')


Now let's see what the data looks like.

In [6]:
pd.set_option('display.max_columns', None)    # shows all columns
print(df.head())

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country  income  
0          2174             0              40   United-States   <=50

Just like the column names, the categorical labels also contain characters like dashes and spaces. The dashes will be replaces by underscores and the spaces will be stripped off.

In [7]:
df[df.select_dtypes('O').columns] = df[df.select_dtypes('O').columns].apply(
    lambda x: x.str.strip(' ').str.replace('-', '_')
)

In section 1, we saw that there were no missing values. We have to check for duplicates as well.

In [8]:
sum(df.duplicated())

24

There are some duplicates which we will remove from the dataset.

In [9]:
df.drop_duplicates(inplace=True)

Now we can dive into the data. We want to see how many unique values there are for each string column.

In [10]:
df.select_dtypes(include='O').nunique().to_frame('# unique values')

,# unique values
workclass,9
education,16
marital_status,7
occupation,15
relationship,6
race,5
sex,2
native_country,42
income,2


There are two binary columns: the last one containing the income, which we already knew and a`sex`. The other non-numerical columns all have more than two unique values. We can create dummies for these two binary columns if there is a need for it in further analysis.

### 2.1 Education

We're going through the categorical columns one by one starting with education. There are two columns for education: a numerical and a textual one.

In [11]:
df['education'].value_counts()

education
HS_grad         10494
Some_college     7282
Bachelors        5353
Masters          1722
Assoc_voc        1382
11th             1175
Assoc_acdm       1067
10th              933
7th_8th           645
Prof_school       576
9th               514
12th              433
Doctorate         413
5th_6th           332
1st_4th           166
Preschool          50
Name: count, dtype: int64

Let's match the two columns to see if it all makes sense.

In [12]:
df[['education', 'education_num']].drop_duplicates().sort_values('education_num').set_index('education_num')

,education
education_num,
1,Preschool
2,1st_4th
3,5th_6th
4,7th_8th
5,9th
6,10th
7,11th
8,12th
9,HS_grad


So the `education_num` works as we would want it to.

### 2.2 Workclass

In [13]:
df['workclass'].value_counts()

workclass
Private             22673
Self_emp_not_inc     2540
Local_gov            2093
?                    1836
State_gov            1298
Self_emp_inc         1116
Federal_gov           960
Without_pay            14
Never_worked            7
Name: count, dtype: int64

We'll replace *'?'* by *'Unknown'*.

In [14]:
df['workclass'] = df['workclass'].str.replace('?', 'Unknown')

## 2.3 Marital Status

In [15]:
df['marital_status'].value_counts()

marital_status
Married_civ_spouse       14970
Never_married            10667
Divorced                  4441
Separated                 1025
Widowed                    993
Married_spouse_absent      418
Married_AF_spouse           23
Name: count, dtype: int64

Nothing out of place here.

### 2.4 Relationship

In [16]:
df['relationship'].value_counts()

relationship
Husband           13187
Not_in_family      8292
Own_child          5064
Unmarried          3445
Wife               1568
Other_relative      981
Name: count, dtype: int64

Let's see if the genders match the martial status.

In [17]:
df.loc[df['relationship'] == 'Husband', 'sex'].value_counts()

sex
Male      13186
Female        1
Name: count, dtype: int64

In [18]:
df.loc[df['relationship'] == 'Wife', 'sex'].value_counts()

sex
Female    1566
Male         2
Name: count, dtype: int64

In [19]:
df.loc[(df['relationship'] == 'Husband') & (df['sex'] == 'Female')]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
7109,34,Private,175878,HS_grad,9,Married_civ_spouse,Sales,Husband,White,Female,0,0,40,United_States,<=50K


In [20]:
df.loc[(df['relationship'] == 'Wife') & (df['sex'] == 'Male')]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
575,29,Private,350162,Bachelors,13,Married_civ_spouse,Exec_managerial,Wife,White,Male,0,0,40,United_States,>50K
27141,36,Private,74791,Bachelors,13,Married_civ_spouse,Sales,Wife,White,Male,0,0,60,?,<=50K


There is clearly an inconsistency here. Husbands are not female and wives are not male. They are all married (to civilians), so they have to be either a husband or a wife. Since there are only two options for sex and multiple for relationship, we will make the educated guess that the sex is correctly documented and the mistake is made in the relationship column.

In [21]:
df.loc[(df['relationship'] == 'Husband') & (df['sex'] == 'Female'), 'relationship'] = 'Wife'
df.loc[(df['relationship'] == 'Wife') & (df['sex'] == 'Male'), 'relationship'] = 'Husband'

In [22]:
df.loc[df['relationship'] == 'Husband', 'marital_status'].value_counts()

marital_status
Married_civ_spouse    13179
Married_AF_spouse         9
Name: count, dtype: int64

In [23]:
df.loc[df['relationship'] == 'Wife', 'marital_status'].value_counts()

marital_status
Married_civ_spouse    1555
Married_AF_spouse       12
Name: count, dtype: int64

All husbands and wives are married.

In [24]:
df.loc[df['relationship'] == 'Unmarried', 'marital_status'].value_counts()

marital_status
Divorced                 1600
Never_married             881
Separated                 451
Widowed                   383
Married_spouse_absent     130
Name: count, dtype: int64

Interestingly, married people with an absent spouse are considered to be unmarried.

In [25]:
df.loc[df['relationship'] == 'Own_child', 'marital_status'].value_counts()

marital_status
Never_married            4481
Divorced                  328
Separated                  99
Married_civ_spouse         95
Married_spouse_absent      45
Widowed                    15
Married_AF_spouse           1
Name: count, dtype: int64

There are quite some people that are married, but live with their parents regardless. The parents are the heads of the household, so the relationship is set to `Own_child`.

In [26]:
df.loc[df['relationship'] == 'Own_child', 'age'].describe()

count    5064.000000
mean       24.832148
std         8.117585
min        17.000000
25%        19.000000
50%        22.000000
75%        27.000000
max        90.000000
Name: age, dtype: float64

There seems to be a 90 year old that lives with their parent(s), which would be remarkable if true.

In [27]:
df.loc[(df['relationship'] == 'Own_child') & (df['age'] == 90)]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
2891,90,Private,171956,Some_college,10,Separated,Adm_clerical,Own_child,White,Female,0,0,40,Puerto_Rico,<=50K
4070,90,Private,313986,11th,7,Never_married,Handlers_cleaners,Own_child,White,Male,0,0,40,United_States,<=50K
11512,90,Private,87285,HS_grad,9,Never_married,Other_service,Own_child,White,Female,0,0,24,United_States,<=50K
12451,90,Unknown,225063,Some_college,10,Never_married,?,Own_child,Asian_Pac_Islander,Male,0,0,10,South,<=50K
18413,90,Private,313749,Bachelors,13,Never_married,Prof_specialty,Own_child,White,Female,0,0,10,United_States,<=50K
18832,90,Private,115306,Masters,14,Never_married,Exec_managerial,Own_child,White,Female,0,0,40,United_States,<=50K


These 90 year olds are supposedly also still working, half of them fulltime even. It is possible, of course, just not very plausible.

In [28]:
df.loc[df['age'] > 70, 'hours_per_week'].describe()

count    539.000000
mean      26.662338
std       16.583715
min        1.000000
25%       14.000000
50%       25.000000
75%       40.000000
max       99.000000
Name: hours_per_week, dtype: float64

Most people seem to be still working past the age of 70. Half of them works at least 25 hours a week. In further analysis, we need to keep this in mind. Something might be going on here that may not be explained by the data.

### 2.5 Occupation

In [29]:
df['occupation'].value_counts()

occupation
Prof_specialty       4136
Craft_repair         4094
Exec_managerial      4065
Adm_clerical         3768
Sales                3650
Other_service        3291
Machine_op_inspct    2000
?                    1843
Transport_moving     1597
Handlers_cleaners    1369
Farming_fishing       992
Tech_support          927
Protective_serv       649
Priv_house_serv       147
Armed_Forces            9
Name: count, dtype: int64

We will replace *'?'* by *'Unknown'*.

In [30]:
df['occupation'] = df['occupation'].str.replace('?', 'Unknown')

### 2.6 Ethnicity

In [31]:
df['race'].value_counts()

race
White                 27795
Black                  3122
Asian_Pac_Islander     1038
Amer_Indian_Eskimo      311
Other                   271
Name: count, dtype: int64

No strange labels or numbers here.

### 2.7 Native Country

In [32]:
df['native_country'].value_counts()

native_country
United_States                 29153
Mexico                          639
?                               582
Philippines                     198
Germany                         137
Canada                          121
Puerto_Rico                     114
El_Salvador                     106
India                           100
Cuba                             95
England                          90
Jamaica                          81
South                            80
China                            75
Italy                            73
Dominican_Republic               70
Vietnam                          67
Guatemala                        62
Japan                            62
Poland                           60
Columbia                         59
Taiwan                           51
Haiti                            44
Iran                             43
Portugal                         37
Nicaragua                        34
Peru                             31
France       

We will replace *'?'* by *'Unknown'*.

In [33]:
df['native_country'] = df['native_country'].str.replace('?', 'Unknown')

There are some typos in the labels:
- Columbia instead of Colombia;
- Holand instead of Holland (which is incorrect anyway, so this should just be Netherlands);
- South (we have to look into the data more to see what is meant here, there are multiple countries whose names start with South);
- Hong instead of Hong Kong;
- Trinadad instead of Trinidad.

Also, Yugoslavia doesn't exist anymore. However, it's hard to determine from which specific country the people are, so we will just keep it as is. 

We will change `Trinidad&Tobago` into `Trinidad_Tobago` to make sure we won't run into any issues with special characters. We will shorten `Outlying_US(Guam_USVI_etc)` into `Outlying_US`.

Let's first check what `South` is supposed to be.

In [34]:
df.loc[df['native_country'] == 'South', 'race'].value_counts()

race
Asian_Pac_Islander    77
Amer_Indian_Eskimo     2
White                  1
Name: count, dtype: int64

Most people from this country are Asian/Pacific Islander, which makes it safe to assume the country is South Korea.

In [35]:
old_labels = ['Columbia', 'Holand_Netherlands', 'South', 'Hong', 'Trinadad&Tobago', 'Outlying_US(Guam_USVI_etc)']
new_labels = ['Colombia', 'Netherlands', 'South_Korea', 'Hong_Kong', 'Trinidad_Tobago', 'Outlying_US']

In [36]:
for i in range(len(old_labels)):
    df['native_country'] = df['native_country'].str.replace(old_labels[i], new_labels[i])

## 3. Summary

We checked for duplicates, missing values, typos and other inconsistencies in the data and corrected them where possible.

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32537 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32537 non-null  int64 
 1   workclass       32537 non-null  object
 2   fnlwgt          32537 non-null  int64 
 3   education       32537 non-null  object
 4   education_num   32537 non-null  int64 
 5   marital_status  32537 non-null  object
 6   occupation      32537 non-null  object
 7   relationship    32537 non-null  object
 8   race            32537 non-null  object
 9   sex             32537 non-null  object
 10  capital_gain    32537 non-null  int64 
 11  capital_loss    32537 non-null  int64 
 12  hours_per_week  32537 non-null  int64 
 13  native_country  32537 non-null  object
 14  income          32537 non-null  object
dtypes: int64(6), object(9)
memory usage: 4.0+ MB


We export the preprocessed data into a csv file so we can work on it in the next part.

In [38]:
df.to_csv('preprocessed_data.csv', index=False)